In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/train-en-ms.tar.gz
# !tar -zxf train-en-ms.tar.gz
# !rm train-en-ms.tar.gz

In [2]:
# !pip3 install pyenchant

In [3]:
# import enchant
# d = enchant.Dict("en_US")
# d.check("Hello")

In [4]:
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/normalization/en-lexicon/en-social-media-lexicon.json
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/normalization/en-lexicon/en-social-media-lexicon-v2.json

In [5]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [6]:
!ls -lha train-en

ls: cannot access 'train-en': No such file or directory


In [7]:
with open('/home/husein/translation/zsm_Latn.dev') as fopen:
    right = fopen.read().split('\n')
    
with open('/home/husein/translation/eng_Latn.dev') as fopen:
    left = fopen.read().split('\n')

In [8]:
import malaya

/home/husein/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [9]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from collections import defaultdict
import re
import random
import json

In [10]:
with open('en-social-media-lexicon.json') as fopen:
    en_lexicon = json.load(fopen)
    
with open('en-social-media-lexicon-v2.json') as fopen:
    en_lexicon_v2 = json.load(fopen)

In [11]:
compound_normalizer = defaultdict(list)
normalizer = defaultdict(list)

for k, v in en_lexicon.items():
    if not len(v):
        continue
    if len(k.split()) > 1:
        compound_normalizer[k].extend(v)
    else:
        normalizer[k].extend(v)

for k, v in en_lexicon_v2.items():
    if not len(v):
        continue
    if len(k.split()) > 1:
        compound_normalizer[k].extend(v)
    else:
        normalizer[k].extend(v)
        
compound_normalizer_regex = (
    '(?:' + '|'.join(list(compound_normalizer.keys())) + ')'
)

In [12]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_


def replace_shortword(word, rules = normalizer):
    left, right, word_ = strip_punct(word)
    word_ = word_[len(left):]
    lower_word_ = word_.lower()
    if lower_word_ in rules:
        word_ = case_of(word_)(random.choice(rules[lower_word_]))
        word_ = f'{left}{word_}{right}'
        return word_
    else:
        return word
    
strip_punct('counters,')

('', ',', 'counters')

In [13]:
# def _replace_compound(string, 
#                       rules_regex = rules_compound_normalizer_regex, 
#                       rules = rev_rules_compound_normalizer):
#     results = re.findall(rules_regex, string, flags=re.IGNORECASE
#     )
#     for r in results:
#         try:
#             string = string.replace(r, random.choice(rules[r.lower()]))
#         except:
#             pass
#     return string

def _replace_compound(string, rules = compound_normalizer):
    for k in list(rules.keys()):
        results = [(m.start(0), m.end(0)) for m in re.finditer(k, string, flags=re.IGNORECASE)]
        for r in results:
            sub = string[r[0]: r[1]]
            try:
                replaced = random.choice(rules[sub.lower()])
                if replaced:
                    if r[1] < len(string) and string[r[1]] != ' ':
                        continue
                    if r[0] - 1 > len(string) and string[r[0] - 1] != ' ':
                        continue

                    sub = case_of(sub)(replaced)
                    string = string[:r[0]] + sub + string[r[1]:]
            except:
                pass
    return string

In [14]:
_replace_compound('i like, text to cell', rules = compound_normalizer)

'i like, ttc'

In [15]:
string = 'i am so beautiful"'
[replace_shortword(word) for word in string.split()]

['im', 'amt', 'som', 'buitfull"']

In [16]:
[replace_shortword(word) for word in left[1].split()]

['Leed',
 'rea',
 'sayy',
 'thia',
 'many',
 'bing',
 'erly',
 'detection',
 'af',
 'cancer,',
 'tuberculosis,',
 'HIV',
 'aand',
 'malaria',
 'qu',
 'patiants',
 'i',
 'low-income',
 'contries,',
 'whereeee',
 'dah',
 'survival',
 'rates',
 'fur',
 'illnesses',
 'sach',
 'saw',
 'brest',
 'cancer',
 'came*2',
 'de',
 'haff',
 'thsoe',
 'afoh',
 'recher',
 'contries.']

In [17]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 5, max_replace = 10):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        try:
            if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
                continue
            elif splitted_left[l].isupper() or splitted_right[r].isupper():
                continue
            elif splitted_left[l] == splitted_right[r]:
                continue
            elif splitted_left[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
                continue
            else:
                selected_alignment.append((l, r))
        except:
            pass
    
    try:
        count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
        selected = random.sample(selected_alignment, count_replace)
        for s in selected:
            splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])

        return ' '.join(splitted_left)
    
    except:
        return ' '.join(splitted_left)

In [18]:
eflomal = malaya.alignment.en_ms.eflomal()

In [19]:
# random replace alignment
# random replace compound
# random replace word

In [20]:
random.random()

0.05309057329529965

In [21]:
malaya.augmentation.socialmedia_form('Saya')

['Saye', 'Sayak']

In [22]:
malaya.augmentation.vowel_alternate('saya')

'saya'

In [23]:
fast_text = malaya.language_detection.fasttext()

In [24]:
import copy

In [25]:
consonants = 'bcdfghjklmnpqrstvwxyz'

def augment(left, right, p_replace_alignment = 0.4, p_replace_shortword = 0.7, p_vowel_alternate = 0.7):
    
    if random.random() > p_replace_alignment:
        alignment = eflomal.align([left], [right])['forward'][0]
        left = random_replace_alignment(left, right, alignment)
    
    left = _replace_compound(left, rules = copy.deepcopy(compound_normalizer))
    left = [(replace_shortword(word, rules = normalizer), False) if random.random() > p_replace_shortword else (word, True) for word in left.split()]
    left_ = []
    for l in left:
        if _is_number_regex(l[0].replace(',', '').replace('.', '')):
            left_.append(l[0])
            continue
        if l[0].isupper():
            left_.append(l[0])
            continue
        if l[0].istitle():
            left_.append(l[0])
            continue
        
        if l[1]:
            s = l[0]
                
            if random.random() > p_vowel_alternate:
                try:
                    s = malaya.augmentation.vowel_alternate(s)
                except:
                    pass
        else:
            s = l[0]
        
        left_.append(case_of(l[0])(s))
    return ' '.join(left_)
    
    return left

In [26]:
malaya.augmentation.vowel_alternate('like')

'like'

In [27]:
%%time

augment(left[0], right[0])

CPU times: user 1.12 ms, sys: 203 µs, total: 1.32 ms
Wall time: 1.32 ms


'On Mon, scc fraom thee Stanford Universit Shcoole of Medacine announced the invention of a new diagnostic tool that kan sort cells by type: a tiney printable chip that ca be manufactured using standard inkjet printers foow possibly about one U.S. cnt each.'

In [28]:
from tqdm import tqdm

new_left, new_right = [], []
for i in tqdm(range(len(left))):
    if len(left[i].split()) > 100 or len(right[i].split()) > 100:
        continue
    langs = fast_text.predict([left[i], right[i]])
    if langs[0] == 'malay':
        continue
    if langs[1] in ['eng', 'ind']:
        continue
    try:
        new_left_ = augment(left[i], right[i])
        if new_left_ != left[i]:
            new_left.append(new_left_)
            new_right.append(right[i])
    except Exception as e:
        pass

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 998/998 [01:46<00:00,  9.40it/s]


In [29]:
len(new_left), len(new_right)

(949, 949)

In [30]:
import json

with open('augmented-eng_Latn-zsm_Latn-noisy-en-ms-dev-set-v2.json', 'w') as fopen:
    json.dump({'ms': new_right, 'en': new_left}, fopen)